In [10]:
from qiskit import IBMQ
IBMQ.save_account('')
import numpy as np
from numpy import linalg
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit import execute, IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from tools import set_parameters
from launcher import Launcher
import cmath
import math
parameters = {}
parameters['backend_name'] = 'qasm_simulator'
parameters['shots'] = 100
launcher = Launcher(**{item: parameters[item] for item in ['backend_name', 'shots']})
#ROTATION
def ConditionedRotation(angle, controllingQbit, subjectedQbit, circuit, quantReg):
  
    circuit.u3(angle, 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])
    circuit.cx(quantReg[controllingQbit], quantReg[subjectedQbit])
    circuit.u3( ( - angle ), 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])
    
    return
#exp(iA)
def summ(x, y):
    return np.array(x)+np.array(y)
def multm(x, y):
    return np.array(x).dot(np.array(y))
def multmn(z, k):
    return k*np.array(z)
def poweia(x):
    j = complex(0, 1)
    c = multmn(x, j)
    u = [[0 for i in range(n - 1)] for j in range(n)]
    for i in range(n):
        u[i].insert(i, 1)
    a = u
    for i in range(1, 100):
        u = multmn(multm(u, c), 1/i)
        a = summ(a, u)
    return a
t = 0.25
#A [2],[1],[1],[2]
n = int(input())
x = []
for i in range(n):
    u = list(map(float, input().split()))
    x.append(u)
#U     
y = poweia(2*np.pi*t*np.array(x))
for i in y:
    print(*i, end = '\n')
if n == 2: 
    a = 0.5*cmath.phase(y[0][0]*y[1][1]) ##alf
    d = 2*math.asin(abs(y[1][0])) ##gamma

    b = cmath.phase(y[1][0]/y[0][0])  ##betta
    c = cmath.phase(y[1][1]/y[0][0])-cmath.phase(y[1][0]/y[0][0]) ##delta

#U^2
y_2 = np.array(y).dot(np.array(y))
for i in y_2:
    print(*i, end = '\n')    
if n == 2: 
    a1 = 0.5*cmath.phase(y_2[0][0]*y_2[1][1]) ##alf1
    d1 = 2*math.atan2(abs(y_2[1][0]/y_2[0][0]),1) + 2*np.pi ##gamma1
    b1 = cmath.phase(y_2[1][0]/y_2[0][0])  ##betta1
    c1 = cmath.phase(y_2[1][1]/y_2[0][0])-cmath.phase(y_2[1][0]/y_2[0][0]) ##delta1
solution = np.array([math.sqrt(0.5), math.sqrt(0.5)])
solution_T = solution.transpose()
# Define the Quantum and Classical Registers
qr = QuantumRegister(5)
cr = ClassicalRegister(5)
# Build the circuits
circuit = QuantumCircuit(qr, cr)
#b
circuit.h(qr[4])

#U
circuit.h(qr[3])
circuit.u1((d - b)/2, qr[4]) #C
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u3(-d/2, 0, -(d + b)/2, qr[4]) #B
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u3(d/2,b, 0, qr[4]) #A
circuit.u1(a, qr[3]) #a

circuit.h(qr[2])
circuit.u1((d1 - b1)/2, qr[4]) #C
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u3(-d1/2, 0, -(d1 + b1)/2, qr[4]) #B
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u3(d1/2,b1, 0, qr[4]) #A
circuit.u1(a1, qr[2]) #a
#FT    
circuit.h(qr[3])
circuit.u1(-(np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1(-(np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1(np.pi/2, qr[3])
circuit.u1(-(np.pi/4), qr[2])
circuit.h(qr[2])    
#R
ConditionedRotation(np.pi/4, 2, 1, circuit, qr)
ConditionedRotation(np.pi/8, 3, 1, circuit, qr)
#FT^-1
circuit.h(qr[2]) 
circuit.u1((np.pi/4), qr[2])
circuit.u1(-np.pi/2, qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1((np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1((np.pi/4), qr[3])
circuit.h(qr[3])
#U^-1

circuit.u1(-a1, qr[2]) #a
circuit.u3(-d1/2,-b1, 0, qr[4]) #A
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u3(d1/2, 0, (d1 + b1)/2, qr[4]) #B
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u1(-(d1 - b1)/2, qr[4]) #C
circuit.h(qr[2])

circuit.u1(-a, qr[3]) #a
circuit.u3(-d/2,-b, 0, qr[4]) #A
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u3(d/2, 0, (d + b)/2, qr[4]) #B
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u1(-(d - b)/2, qr[4]) #C
circuit.h(qr[3])
# meas_qubits - кубиты, использующиеся для томографии. Порядок важен (для [0, 1] и [1, 0] ответ будет разным)
exp_solution_4 = launcher.run([circuit], meas_qubits=[4,1])[0]
print(exp_solution_4)
exp_solution_2 = np.array([[exp_solution_4[0][0] + exp_solution_4[1][1], exp_solution_4[2][0] + exp_solution_4[3][1]],
                          [exp_solution_4[0][2]+exp_solution_4[1][3],exp_solution_4[2][2] + exp_solution_4[3][3]]])
prob = solution @ exp_solution_2 @ solution_T
print(prob)


2
2 1
1 2
(8.943923166284532e-16-1.086033874176431e-15j) (-1.31051831842631e-15-0.999999999999999j)
(-1.3937850452731478e-15-0.999999999999999j) (-8.817612878040172e-16+9.124217736400364e-16j)
(-0.999999999999998+2.704303363699453e-15j) (-1.736121005363943e-16-1.2631028824435714e-17j)
(-1.736121005363943e-16-1.2631028824435616e-17j) (-0.999999999999998+2.7043033636994535e-15j)


HTML(value="<p style='font-size:16px;'>Job Status: job is actively running </p>")

[[ 0.09184993+0.j          0.00989915-0.02563531j  0.05385316-0.02103687j
  -0.05059005+0.00100282j]
 [ 0.00989915+0.02563531j  0.11572055+0.j          0.0315406 +0.02076122j
   0.0076852 +0.0388715j ]
 [ 0.05385316+0.02103687j  0.0315406 -0.02076122j  0.33928467+0.j
  -0.01224646+0.03496105j]
 [-0.05059005-0.00100282j  0.0076852 -0.0388715j  -0.01224646-0.03496105j
   0.45314484+0.j        ]]
(0.5615383592612344+0j)


In [3]:
circuit.draw()